<a href="https://colab.research.google.com/github/qingxinxia/OpenPackChallenge2025/blob/main/1.Data%20Augmentation%20Algorithm%20with%20HAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Virtual Data Generation for Complex Industrial Activity Recognition



This notebook has been designed for the 7th Factory Work Activity Recognition Challenge competition with the aim of Activity Recognition using REAL Accelerometer from OpenPack dataset and GENERATED Accelerometer created by participants.

If you have any questions, please feel free to email qingxinxia@hkust-gz.edu.cn with the subject Factory Work Activity Recognition Challenge.

About this dataset and challenge -> https://abc-research.github.io/challenge2025/

This notebook was prepared by Qingxin Xia, Kei Tanigaki and Yoshimura Naoya.

---

# 工場作業行動認識のための仮想データ生成
本ノートブックは、第7回工場作業行動認識チャレンジのために設計されました。本チャレンジでは、OpenPackデータセット作成に用いられた実際の加速度計と参加者によって生成された加速度センサデータを使用しています。

ご質問がある場合は、件名を「工場作業行動認識チャレンジ」として、qingxinxia@hkust-gz.edu.cnまでお気軽にメールしてください。

このデータセットとチャレンジについて -> https://abc-research.github.io/challenge2025/

このノートブックはQingxin Xia, Kei TanigakiとYoshimura Naoyaによって準備されました。

# 1. Preparation

---

# 1. 準備する

## 1.1 Mount Drive

This tutorial is made in Google Colab. So, first we need to connect the Google Drive to access the data. You can directly add folder path to access the data.


---

## 1.1 Mount Drive

本チュートリアルは Google Colab で作成されています。
そのため、まず Google ドライブに接続してデータにアクセスする必要があります。
フォルダーパスを追加することでドライブへのアクセスが可能になります。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 1.2 Load Necessary Libraries and Prepare Environment

---
## 1.2 必要なライブラリをロードして環境を準備する


In [ ]:
import os
import random
import zipfile
import requests
import numpy as np
import pandas as pd
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.nn.functional as F
import math
from scipy.interpolate import CubicSpline  # for warping
from einops import rearrange, repeat
from torch.autograd import Variable
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
print(device)

## 1.3 Fixed Part

Participants can change the splits portion and random seed to measure the robustness of their algorithms. When we evaluate the code, selected_columns and new_columns will not change. However, the split and random seed will change.


---
## 1.3 修正不可能な箇所

参加者は、splitとrandom seedを変更して、アルゴリズムの堅牢性を測定できます。我々がコードを評価する際には、selected_columns と new_columns は変更されませんが、split と random seed は変更されます。



In [ ]:
splits = [0.7, 0.1, 0.2]
print('Randomly Split the real dataset into train, validation and test sets: %s'%str(splits))

selected_columns = ['atr01/acc_x','atr01/acc_y','atr01/acc_z','atr02/acc_x','atr02/acc_y','atr02/acc_z','timestamp','operation']
print('Select acceleration data of both wrists: %s'%selected_columns)

new_columns = selected_columns[:6] + [selected_columns[-1]]
print('Data for train, validation, and test: %s'%new_columns)

def set_random_seed(seed):
    # Set seed for Python's random module
    random.seed(seed)

    # Set seed for NumPy
    np.random.seed(seed)

    # Set seed for PyTorch
    torch.manual_seed(seed)

    # If using CUDA, set seed for GPU as well
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # For multi-GPU setups

# Set a fixed random seed
seed_value = 2025
set_random_seed(seed_value)

## 1.4 Create Folders

Firstly, a folder is created to save OpenPack dataset: '/data/real/'.

Participants can download OpenPack dataset by themselves. The data should be placed at: '/data/real/'.

Another folder '/data/virtual/' will be created to save generated data.
 *(Note that, the size of this 'virtual' folder is limited to 500MB.)*

---
## 1.4 フォルダーの作成

まず、OpenPack データセットを保存するためのフォルダー「/data/real/」が作成されます。

参加者は OpenPack データセットを自分でダウンロードできます。データは「/data/real/」に配置する必要があります。

生成されたデータを保存するために、別のフォルダー「/data/virtual/」が作成されます。

*(この「virtual」フォルダーのサイズは500MBに制限されていることに注意してください。)*


In [ ]:
realpath = r'/data/real'
virtpath = r'/data/virtual'
rootdir = r'/Virtual_Data_Generation/'  # replace with your project path
real_directory = rootdir + realpath
virt_directory = rootdir + virtpath

# Create the directory
os.makedirs(real_directory, exist_ok=True)
print(f"Directory '{realpath}' created successfully.")

# Create the directory
os.makedirs(virt_directory, exist_ok=True)
print(f"Directory '{virtpath}' created successfully.")


## 1.5 Download data from Zenodo

Participants can also use this code to download the OpenPack dataset.


---
## 1.5 Zenodo からデータをダウンロード

参加者はこのコードを使用して OpenPack データセットをダウンロードすることができます。


In [ ]:
# Construct the URL to the Zenodo API
api_url = f"https://zenodo.org/records/11059235"

# Send a request to the Zenodo API
response = requests.get(api_url)
response.raise_for_status()  # Check for HTTP errors

# # Parse the JSON response
# data = response.json()

# Extract the file information
download_url = f"https://zenodo.org/records/11059235/files/imu-with-operation-action-labels.zip?download=1"

# Download the file
file_response = requests.get(download_url)
file_response.raise_for_status()  # Check for HTTP errors

# Save the file
file_path = os.path.join(real_directory, 'imu-with-operation-action-labels.zip')
with open(file_path, 'wb') as f:
    f.write(file_response.content)

print(f"Downloaded to {file_path}")


## 1.6 Unzip OpenPack dataset

After placing the OpenPack dataset at the '/data/real/' folder, unzip the files and delete the zip files.

---
## 1.6 OpenPack データセットを解凍する

OpenPack データセットを '/data/real/' フォルダに配置した後、ファイルを解凍し、zip ファイルを削除します。


In [ ]:
# Iterate over all files in the directory
for filename in os.listdir(real_directory):
    # Construct full file path
    file_path = os.path.join(real_directory, filename)

    # Check if the file is a zip file
    if filename.endswith('.zip'):
        # Open the zip file
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            # Extract all contents of the zip file into the directory
            zip_ref.extractall(file_path[:-4])
            print(f"Extracted: {filename}")

        # Delete the zip file
        os.remove(file_path)
        print(f"Deleted: {filename}")

print("All zip files have been processed.")

# 2. Use Real Data to Generate Virtual Data

Firstly, we randomly split the real data into a training set, validation set, and test set according to a specific ratio. Participants can then use the training set to generate virtual data. Finally, we train the network using both the training set and the virtual data, and we calculate the F1 score on the test set.

The following code provides an example of generating virtual data from the training set. Note that: (1) The model structure is fixed and unchanged. (2) The split ratio for the training and test sets, as well as the random seed, will differ from the current settings, which requires the virtual data generation algorithm to be robust against varying data. (3) Participants are free to design data generation algorithms and save them to a specified path: '/data/virtual/', but the size of the virtual data is limited to 500MB.

---

# 2. 実データを使用して仮想データを生成する

まず、実データを特定の比率に従ってトレーニングセット、検証セット、テストセットにランダムに分割します。参加者はトレーニングセットを使用して仮想データを生成できます。最後に、トレーニングセットと仮想データの両方を使用してネットワークをトレーニングし、テストセットの F1 スコアを計算します。

次のコードは、トレーニング セットから仮想データを生成する例を示しています。次の点に注意してください。

(1) モデル構造は固定されており、変更することはできません。

(2) トレーニングセットとテストセットの分割比率、およびランダムシードは現在の設定とは異なります。そのため、仮想データ生成アルゴリズムは変化するデータに対して堅牢である必要があります。

(3) 参加者はデータ生成アルゴリズムを自由に設計し、指定されたパス「/data/virtual/」に保存できますが、仮想データのサイズは500MBに制限されます。

## 2.1 Assign train users, validation users, and test users

In the OpenPack dataset, U0xxx corresponds to user IDs, and S0xxx corresponds to different experiment settings.

In this Challenge, we will only select training (real) and test data from S0100.

---
## 2.1 トレーニング ユーザー、検証ユーザー、テスト ユーザーを割り当てる

OpenPack データセットでは、U0xxx はユーザー ID に対応し、S0xxx はさまざまな実験設定に対応します。

このチャレンジでは、S0100 からトレーニング (実際の) データとテスト データのみを選択します。


## 2.2 Filter out un-used data

---

## 2.2 未使用のデータを除外する

In [ ]:
user_paths = {}
for root, dirs, files in os.walk(real_directory):
    for file in files:
        if file.endswith('S0100.csv'):
            user_paths[file[:-10]] = os.path.join(root, file)
        else:
          os.remove(os.path.join(root, file))  # remove unused data
for u, d in user_paths.items():
    print('%s at: %s'% (u,d))

## 2.3 Split users to train, validation, and test sets

---

## 2.3 ユーザーをトレーニング、検証、テストセットに分割する

In [ ]:
userIDs = list(user_paths.keys())

# Shuffle the list to ensure randomness
random.shuffle(userIDs)

# Calculate the split indices
total_length = len(userIDs)
train_size = int(total_length * splits[0])  # 70% of 10
val_size = int(total_length * splits[1])  # 10% of 10
test_size = total_length - train_size - val_size  # 20% of 10

# Split the list according to the calculated sizes
train_users = np.sort(userIDs[:train_size])      # First 70%
val_users = np.sort(userIDs[train_size:train_size + val_size])  # Next 10%
test_users = np.sort(userIDs[train_size + val_size:])  # Last 20%

print('Training set: %s'%train_users)
print('Validation set: %s'%val_users)
print('Test set: %s'%test_users)


## 2.4 Load data according to userIDs

Load data of every user as dataframe.
Use acceleration data of both wrists only;
Use operation label.

---
## 2.4 ユーザーID に従ってデータをロードします

すべてのユーザーのデータをデータフレームとしてロードします。
両手首の加速度データのみを使用します。
操作ラベルを使用します。


In [ ]:
# selected_columns = ['atr01/acc_x','atr01/acc_y','atr01/acc_z','atr02/acc_x','atr02/acc_y','atr02/acc_z','timestamp','operation']
train_data_dict = {}
for u in train_users:
    # Load the CSV file with only the selected columns
    train_data_dict[u] = pd.read_csv(user_paths[u], usecols=selected_columns)

val_data_dict = {}
for u in val_users:
    # Load the CSV file with only the selected columns
    val_data_dict[u] = pd.read_csv(user_paths[u], usecols=selected_columns)

test_data_dict = {}
for u in test_users:
    # Load the CSV file with only the selected columns
    test_data_dict[u] = pd.read_csv(user_paths[u], usecols=selected_columns)

## 2.6 Virtual Data Generation

In [ ]:
import os
import glob

def delete_csv_files(folder_path):
    """
    Deletes all CSV files in the specified folder.

    Args:
        folder_path (str): The path to the folder containing the CSV files.
    """
    csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
    for file_path in csv_files:
        try:
            os.remove(file_path)
            print(f"Deleted: {file_path}")
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")


In [ ]:
import os

def get_folder_size(folder):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if os.path.exists(fp):  # Skip broken symlinks
                total_size += os.path.getsize(fp)
    return total_size

def check_virtual_volume(rootdir, virtpath, limit_mb=500):
    """
    Check if the total size of the virtual data folder stays within limit_mb megabytes.
    
    Parameters:
      rootdir (str): The base root directory.
      virtpath (str): The virtual data folder path relative to rootdir.
      limit_mb (int): The size limit in MB (default is 500 MB).
    """
    virt_directory = rootdir + virtpath
    total_virtual_size = get_folder_size(virt_directory)
    limit = limit_mb * 1024 * 1024  # convert MB to bytes

    if total_virtual_size <= limit:
        # print(f"合計サイズは {total_virtual_size/(1024**2):.2f} MB で、{limit_mb}MB以下です。")
        return True
    else:
        # print(f"合計サイズは {total_virtual_size/(1024**2):.2f} MB で、{limit_mb}MBを超えています。")
        return False

In [ ]:
import os
import uuid
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d, Rbf, PchipInterpolator, Akima1DInterpolator
from scipy.interpolate import splrep, splev
from scipy.signal import savgol_filter
import pywt
from pykalman import KalmanFilter
from sklearn.linear_model import RANSACRegressor
from delete import delete_csv_files
from volumeChecker import check_virtual_volume
from scipy.interpolate import interp1d, CubicSpline
import numpy as np
from scipy.signal.windows import blackmanharris
from scipy.special import gamma
explanatory_columns = ['atr01/acc_x','atr01/acc_y','atr01/acc_z',
                       'atr02/acc_x','atr02/acc_y','atr02/acc_z']
# ---------------------------
# 各 (operation, action) のセグメント長分布を計算
# ---------------------------
segment_length_dist = {}  # キーは (operation, action) タプル
raw_segment_length_dist = {}  # 外れ値除去前の元データも保存しておく

for u, df in train_data_dict.items():
    df_temp = df.copy()
    # operationまたはactionが変化したら新しいグループとする
    df_temp['group'] = ((df_temp['operation'] != df_temp['operation'].shift()) | 
                        (df_temp['action'] != df_temp['action'].shift())).cumsum()
    for _, group_df in df_temp.groupby('group'):
        key = (group_df['operation'].iloc[0], group_df['action'].iloc[0])
        seg_len = len(group_df)
        raw_segment_length_dist.setdefault(key, []).append(seg_len)

# IQRを使用して外れ値を除外する
for key, lengths in raw_segment_length_dist.items():
    lengths_arr = np.array(lengths)
    
    # IQR計算
    q1 = np.percentile(lengths_arr, 25)
    q3 = np.percentile(lengths_arr, 75)
    iqr = q3 - q1
    
    # 外れ値の境界を定義 (一般的に使われる1.5*IQRを使用)
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # 外れ値でないデータのみをフィルタリング
    filtered_lengths = lengths_arr[(lengths_arr >= lower_bound) & (lengths_arr <= upper_bound)]
    
    # 外れ値を除いたデータを保存
    segment_length_dist[key] = filtered_lengths.tolist()

# 各セグメントの統計量を出力（デバッグ用）
for key, lengths in segment_length_dist.items():
    lengths_arr = np.array(lengths)
    raw_lengths = np.array(raw_segment_length_dist[key])
    
    # フィルタリング前後の統計情報を出力
    print(f"Operation {key[0]}, Action {key[1]}: ")
    print(f"  Raw: count={len(raw_lengths)}, mean={raw_lengths.mean():.2f}, median={np.median(raw_lengths):.2f}, std={raw_lengths.std():.2f}")
    print(f"  Filtered: count={len(lengths_arr)}, mean={lengths_arr.mean():.2f}, median={np.median(lengths_arr):.2f}, std={lengths_arr.std():.2f}")
    print(f"  Removed: {len(raw_lengths) - len(lengths_arr)} outliers")

# ---------------------------
# 補助関数の定義
# ---------------------------
def generate_unique_filename(prefix):
    """ユーザID等を接頭辞にしてユニークなファイル名を生成する"""
    return f"{prefix}_{uuid.uuid4().hex}.csv"

# --- 外れ値フィルタリング ---
def filter_moving_average(series, window=5):
    return series.rolling(window, min_periods=1, center=True).mean()

def filter_ransac(series):
    x = np.arange(len(series)).reshape(-1, 1)
    y = series.values
    model = RANSACRegressor()
    model.fit(x, y)
    y_pred = model.predict(x)
    return pd.Series(y_pred, index=series.index)

def filter_wavelet(series, wavelet='db1'):
    coeff = pywt.wavedec(series, wavelet)
    threshold = np.median(np.abs(coeff[-1])) / 0.6745
    new_coeff = [pywt.threshold(c, threshold, mode='soft') for c in coeff]
    rec = pywt.waverec(new_coeff, wavelet)
    rec = rec[:len(series)]
    return pd.Series(rec, index=series.index)

def filter_kalman(series):
    kf = KalmanFilter(initial_state_mean=series.iloc[0], n_dim_obs=1)
    state_means, _ = kf.smooth(series.values)
    return pd.Series(state_means.flatten(), index=series.index)

def filter_savgol(series, window_length=7, polyorder=2):
    if window_length >= len(series):
        window_length = len(series) if len(series) % 2 == 1 else len(series) - 1
    # Ensure polyorder is less than window_length
    if polyorder >= window_length:
        polyorder = window_length - 1
    return pd.Series(savgol_filter(series, window_length=window_length, polyorder=polyorder), index=series.index)

def apply_outlier_filter(series, method='none'):
    if method == 'moving_average':
        return filter_moving_average(series)
    elif method == 'ransac':
        return filter_ransac(series)
    elif method == 'wavelet':
        return filter_wavelet(series)
    elif method == 'kalman':
        return filter_kalman(series)
    elif method == 'savgol':
        return filter_savgol(series)
    elif method == 'none':
        return series
    else:
        return series

# --- 補間手法の定義 ---
def interpolate_linear(x, y, new_x):
    # Replace non-finite values in y with linearly interpolated values if any
    if not np.all(np.isfinite(y)):
        y = pd.Series(y).interpolate(method='linear', limit_direction='both').values
    f = interp1d(x, y, kind='linear', fill_value="extrapolate")
    return f(new_x)

def interpolate_rbf(x, y, new_x, function='multiquadric'):
    # Replace non-finite values in y with linearly interpolated values if any
    if not np.all(np.isfinite(y)):
        y = pd.Series(y).interpolate(method='linear', limit_direction='both').values
    rbf_func = Rbf(x, y, function=function)
    return rbf_func(new_x)

def interpolate_bspline(x, y, new_x, k=3, s=0):
    if len(x) <= k:
        return np.interp(new_x, x, y)
    tck = splrep(x, y, k=k, s=s)
    return splev(new_x, tck)

def interpolate_akima(x, y, new_x):
    # Replace non-finite values in y with linearly interpolated values if any
    if not np.all(np.isfinite(y)):
        y = pd.Series(y).interpolate(method='linear', limit_direction='both').values
    try:
        akima = Akima1DInterpolator(x, y)
        return akima(new_x)
    except ValueError:
        return np.interp(new_x, x, y)

def interpolate_pchip(x, y, new_x):
    # Replace non-finite values in y with linearly interpolated values if any
    if not np.all(np.isfinite(y)):
        y = pd.Series(y).interpolate(method='linear', limit_direction='both').values
    pchip = PchipInterpolator(x, y)
    return pchip(new_x)

def perform_interpolation(x, y, new_x, method='pchip', **kwargs):
    x = np.asarray(x)
    y = np.asarray(y)
    # Filtering NaNなど必要ならここで処理する
    if method == 'interpolate':
        return interpolate_linear(x, y, new_x)
    elif method == 'rbf_inverse':
        return interpolate_rbf(x, y, new_x, function='inverse')
    elif method == 'rbf_multiquadric':
        return interpolate_rbf(x, y, new_x, function='multiquadric')
    elif method == 'rbf_gaussian':
        return interpolate_rbf(x, y, new_x, function='gaussian')
    elif method == 'rbf_linear':
        return interpolate_rbf(x, y, new_x, function='linear')
    elif method == 'rbf_cubic':
        return interpolate_rbf(x, y, new_x, function='cubic')
    elif method == 'bspline':
        return interpolate_bspline(x, y, new_x)
    elif method == 'fft_hann':
        return interpolate_fft(x, y, new_x, window='hann')
    elif method == 'fft_Welch':
        return interpolate_fft(x, y, new_x, window='Welch')
    elif method == 'fft_Blackman-Harris':
        return interpolate_fft(x, y, new_x, window='Blackman-Harris')
    elif method == 'akima':
        return interpolate_akima(x, y, new_x)
    elif method == 'pchip':
        return interpolate_pchip(x, y, new_x)
    elif method == "hermite":
        # Use PCHIP (Piecewise Cubic Hermite Interpolating Polynomial) for Hermite interpolation
        pchip = PchipInterpolator(x, y)
        return pchip(new_x)
    elif method == 'hybrid':
        return hybrid_interpolate(x, y, new_x)
    else:
        return interpolate_linear(x, y, new_x)

def hybrid_interpolate(time, values, new_x, max_linear_gap=5):
    """
    Interpolate missing values (NaNs) in a time series using a hybrid approach:
    - Linear interpolation for gaps shorter or equal to max_linear_gap.
    - Cubic spline interpolation for larger gaps.
    After filling missing values on the original time axis, the result is interpolated onto new_x.
    """
    values = np.asarray(values).copy()
    n = len(values)
    isnan = np.isnan(values)
    if not np.any(isnan):
        if len(new_x) != n:
            return np.interp(new_x, time, values)
        return values
    
    # Find gaps (continuous NaN segments) by scanning through data
    i = 0
    while i < n:
        if np.isnan(values[i]):
            start = i
            while i < n and np.isnan(values[i]):
                i += 1
            end = i
            gap_length = end - start
            if gap_length <= max_linear_gap:
                if start == 0 or end == n:
                    continue
                x0, x1 = time[start-1], time[end]
                y0, y1 = values[start-1], values[end]
                interp_times = time[start:end]
                values[start:end] = y0 + (y1 - y0) * ((interp_times - x0) / (x1 - x0))
            else:
                if start == 0 or end == n:
                    if start == 0 and end < n:
                        values[start:end] = values[end]
                    elif end == n and start > 0:
                        values[start:end] = values[start-1]
                    continue
                idx_before = start - 1
                idx_after = end
                spline_x = time[[idx_before, idx_after]]
                spline_y = values[[idx_before, idx_after]]
                cs = CubicSpline(spline_x, spline_y, bc_type='natural')
                interp_times = time[start:end]
                values[start:end] = cs(interp_times)
        else:
            i += 1
    if len(new_x) != n:
        values = np.interp(new_x, time, values)
    return values
def interpolate_fft(x, y, new_x, window='hann'):
    """
    FFTベースの補間を実施する関数

    Parameters:
      x : array-like
          元のタイムスタンプ（均等サンプリングされている前提）
      y : array-like
          補間対象のデータ系列
      new_x : array-like
          補間後のタイムスタンプ。補間結果は新たに一様な系列として得られるが、
          new_xが元の一様系列と異なる場合、線形補間により最終的な値を算出する。
      window : str, optional
          使用する窓関数。'hann'、'Welch'、'Blackman-Harris'、もしくはその他（デフォルトは'hann'）。
    
    Returns:
      y_final : array-like
          new_xに対応する補間後のデータ系列
    """
    # numpy配列に変換
    x = np.asarray(x)
    y = np.asarray(y)
    new_x = np.asarray(new_x)
    
    N = len(y)
    new_N = len(new_x)
    
    # --- 窓関数の生成 ---
    if window == 'hann':
        win = np.hanning(N)
    elif window == 'Welch':
        # Welch窓はパラボリック窓とも呼ばれる
        n = np.arange(N)
        win = 1 - ((n - (N-1)/2) / ((N-1)/2))**2
    elif window == 'Blackman-Harris':
        win = blackmanharris(N)
    else:
        win = np.ones(N)
    
    # 入力系列に窓を適用
    y_windowed = y * win

    # --- FFTを計算 ---
    Y = np.fft.fft(y_windowed)
    
    # --- ゼロパディングまたはトランケーション ---
    if new_N > N:
        pad = new_N - N
        # FFT係数の左右対称性を保つため、中央にゼロを挿入
        if N % 2 == 0:
            left = Y[:N//2]
            right = Y[N//2:]
        else:
            left = Y[:(N+1)//2]
            right = Y[(N+1)//2:]
        Y_padded = np.concatenate([left, np.zeros(pad, dtype=complex), right])
    else:
        # 補間先のサイズが小さい場合は先頭new_N個を採用（必要に応じた実装変更も可）
        Y_padded = Y[:new_N]
    
    # --- 逆FFTにより時系列信号に戻す ---
    y_interp = np.fft.ifft(Y_padded)
    # 実部を取り、スケーリング（新旧データ数の比率）で調整
    y_interp = np.real(y_interp) * (float(new_N) / N)
    
    # --- FFT補間結果は一様なタイム軸上のデータとなる ---
    t_uniform = np.linspace(x[0], x[-1], new_N)
    
    # new_xが一様なタイム軸と一致しない場合は、線形補間で最終調整
    if not np.allclose(new_x, t_uniform):
        y_final = np.interp(new_x, t_uniform, y_interp)
    else:
        y_final = y_interp

    return y_final


def dtw_align_paths(seq1, seq2, dist=lambda x, y: abs(x - y)):
    n, m = len(seq1), len(seq2)
    dtw = np.full((n+1, m+1), np.inf)
    dtw[0, 0] = 0
    ptr = np.zeros((n+1, m+1, 2), dtype=int)
    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = dist(seq1[i-1], seq2[j-1])
            choices = [dtw[i-1, j], dtw[i, j-1], dtw[i-1, j-1]]
            idx = np.argmin(choices)
            dtw[i, j] = cost + choices[idx]
            if idx == 0:
                ptr[i, j] = [i-1, j]
            elif idx == 1:
                ptr[i, j] = [i, j-1]
            else:
                ptr[i, j] = [i-1, j-1]
    i, j = n, m
    path = []
    while i > 0 or j > 0:
        path.append((i-1, j-1))
        i_prev, j_prev = ptr[i, j]
        i, j = i_prev, j_prev
    path.reverse()
    return path

# ---------------------------
# 仮想データ生成処理
# ---------------------------
def process_csv_files(output_dir, num_timestamps=None,
                      outlier_method='moving_average', 
                      interpolation_method='bspline',
                      round_values=False,
                      distribution_type='normal'):
    """
    指定した出力ディレクトリに、trainデータから生成した仮想データCSVを出力する。
    手順:
      1. train_data_dict内の各CSVデータについて、operationおよびactionの変化ごとにセグメントを抽出
         （セグメント長分布は事前に計算した分布を利用）
      2. 各セグメントのtimestampを先頭0起点に調整
      3. 指定の外れ値除去法で加速度データをフィルタリング
      4. 指定の補間手法で各セグメントを補間し、新しいタイムスタンプ系列にリサンプリング
         - num_timestamps=Trueの場合、各セグメントの長さを、(operation, action)ごとの実データ分布からランダムにサンプリングする
         - num_timestamps=Noneの場合は元のtimestamp系列をそのまま利用
      5. 必要に応じて値を丸め（round_values）
      6. CSVとして出力し、出力ディレクトリの容量がlimit_mb未満なら処理を継続
    
    Args:
        distribution_type: 生成するセグメント長の分布タイプ ('normal', 'gamma', 'exponential')
    """
    # 出力先内のCSVをすべて削除
    delete_csv_files(output_dir)
    # ループ：出力ディレクトリ容量が500MB未満の場合に生成（容量超えたら終了）
    while check_virtual_volume(rootdir, virtpath, limit_mb=500) is True:
        for user, df in train_data_dict.items():
            if check_virtual_volume(rootdir, virtpath, limit_mb=500) is False:
                return None
            df_proc = df[selected_columns].copy()
            # operationまたはactionが変化したら新グループとする
            df_proc['group'] = ((df_proc['operation'] != df_proc['operation'].shift()) |
                                (df_proc['action'] != df_proc['action'].shift())).cumsum()
            processed_groups = []
            for group_id, group_df in df_proc.groupby('group'):
                group_df = group_df.copy()
                # タイムスタンプを先頭0起点に調整
                group_df['timestamp'] = group_df['timestamp'] - group_df['timestamp'].iloc[0]
                
                # 新しいtimestamp系列の生成
                if num_timestamps:
                    op_val = group_df['operation'].iloc[0]
                    act_val = group_df['action'].iloc[0]
                    key = (op_val, act_val)
                    if key in segment_length_dist:
                        durations = np.array(segment_length_dist[key])
                        mean_val = durations.mean()
                        std_val = durations.std()
                        
                        # 選択された分布タイプに基づいてサンプリング
                        if distribution_type == 'normal':
                            current_num = int(np.round(np.random.normal(mean_val, std_val)))
                        elif distribution_type == 'gamma':
                            # ガンマ分布パラメータ: シェイプ k とスケール θ を推定
                            # 正規分布の平均と分散からガンマ分布のパラメータを推定
                            if mean_val > 0 and std_val > 0:
                                shape = (mean_val / std_val) ** 2
                                scale = std_val ** 2 / mean_val
                                current_num = int(np.round(np.random.gamma(shape, scale)))
                            else:
                                current_num = int(mean_val)
                        elif distribution_type == 'exponential':
                            # 指数分布のパラメータ λ = 1/平均
                            if mean_val > 0:
                                scale = mean_val  # 指数分布の scale=1/rate
                                current_num = int(np.round(np.random.exponential(scale)))
                            else:
                                current_num = int(mean_val)
                        elif distribution_type == 'lognormal':
                            # 対数正規分布のパラメータ μ と σ を推定
                            if mean_val > 0 and std_val > 0:
                                var = std_val ** 2
                                mu = np.log(mean_val**2 / np.sqrt(var + mean_val**2))
                                sigma = np.sqrt(np.log(1 + var / mean_val**2))
                                current_num = int(np.round(np.random.lognormal(mu, sigma)))
                            else:
                                current_num = int(mean_val)
                        elif distribution_type == 'weibull':
                            # ワイブル分布のパラメータ k と λ を推定
                            # 形状パラメータ k は1.2（やや右に裾が長い）、スケールパラメータはmean_valを基準に調整
                            if mean_val > 0:
                                k = 1.2  # 形状パラメータ
                                # Γ(1 + 1/k) = ガンマ関数
                                lambda_param = mean_val / gamma(1 + 1/k)
                                current_num = int(np.round(np.random.weibull(k) * lambda_param))
                            else:
                                current_num = int(mean_val)
                        elif distribution_type == 'poisson':
                            # ポアソン分布（整数値を生成する）
                            if mean_val > 0:
                                current_num = np.random.poisson(mean_val)
                            else:
                                current_num = 1
                        else:
                            # デフォルトは正規分布
                            current_num = int(np.round(np.random.normal(mean_val, std_val)))
                            
                        current_num = max(current_num, 1)
                    else:
                        current_num = max(len(group_df), 1)
                    new_timestamps = np.linspace(group_df['timestamp'].min(),
                                                 group_df['timestamp'].max(),
                                                 current_num)
                else:
                    new_timestamps = group_df['timestamp'].values
                
                # 新しいグループDataFrameを作成（補間後の行数 = new_timestampsの長さ）
                new_group_df = pd.DataFrame({'timestamp': new_timestamps})
                # グループ内の行数が1なら、外れ値フィルタや補間処理をスキップ
                if len(group_df) == 1:
                    if num_timestamps is not None:
                        # num_timestamps が指定されていれば、新しいDataFrameを作成する
                        ts_min = group_df['timestamp'].min()
                        ts_max = group_df['timestamp'].max()
                        new_timestamps = np.linspace(ts_min, ts_max, num_timestamps)
                        # 元のデータを複製して新しいDataFrameを作成
                        new_df = pd.DataFrame([group_df.iloc[0].to_dict()] * num_timestamps)
                        new_df['timestamp'] = new_timestamps
                        processed_groups.append(new_df)
                    else:
                        processed_groups.append(group_df)
                    continue
                # 各説明変数に対してフィルタと補間を実施
                for col in explanatory_columns:
                    filtered_series = apply_outlier_filter(group_df[col], method=outlier_method)
                    interp_values = perform_interpolation(group_df['timestamp'].values,
                                                          filtered_series.values,
                                                          new_timestamps,
                                                          method=interpolation_method)
                    new_group_df[col] = interp_values
                # operation列を付与（actionは評価には使わないので省略可）
                new_group_df['operation'] = group_df['operation'].iloc[0]
                processed_groups.append(new_group_df)
            final_df = pd.concat(processed_groups, ignore_index=True)
            if round_values:
                final_df = final_df.round(5)
            if 'timestamp' in final_df.columns:
                final_df = final_df.drop(columns=['timestamp'])
            if 'action' in final_df.columns:
                final_df = final_df.drop(columns=['action'])
            if 'group' in final_df.columns:
                final_df = final_df.drop(columns=['group'])
            output_file = os.path.join(output_dir, generate_unique_filename(user))
            final_df.to_csv(output_file, index=False)
            if check_virtual_volume(rootdir, virtpath, limit_mb=500) is False:
                return None


In [ ]:
def get_folder_size(folder_path):
    total_size = 0
    # Walk through all files and subdirectories in the folder
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            # Get the full file path
            file_path = os.path.join(dirpath, filename)
            # Add the size of each file to the total size
            total_size += os.path.getsize(file_path)
    return total_size

# Example usage
size = get_folder_size(virt_directory)
print(f"The total size of the folder is: {size} bytes")

# 3. Use the Generated Data to Improve HAR Model Performance



---
# 3. 生成されたデータを使用してHARモデルのパフォーマンスを向上させる


## 3.1 Read data and labels from both real and virtual folders


---
## 3.1 実フォルダと仮想フォルダの両方からデータとラベルを読み取る


In [ ]:
# find csv files in 'data/virtual'
virt_paths = []
for root, dirs, files in os.walk(virt_directory):
    for file in files:
        if file.endswith('.csv'):
            virt_paths.append(os.path.join(root, file))
print('Virtual csv file paths are as shown follows:')
virt_paths

In [ ]:
# real and virtual training data

## real data
train_data = []
for u, data in train_data_dict.items():
    train_data.append(data[new_columns].values)
    # print(data[new_columns].values.shape)

## virtual data
for p in virt_paths:
    # Load the CSV file with only the selected columns
    data = pd.read_csv(p, usecols=new_columns)
    train_data.append(data.values)

train_data = np.concatenate(train_data, axis=0)
print('Shape of train data is %s'%str(train_data.shape))

In [ ]:
# validatation and test data
val_data = []
for u, data in val_data_dict.items():
    val_data.append(data[new_columns].values)

test_data = []
for u, data in test_data_dict.items():
    test_data.append(data[new_columns].values)

val_data = np.concatenate(val_data, axis=0)
test_data = np.concatenate(test_data, axis=0)

print('Shape of validation data is %s'%str(val_data.shape))
print('Shape of test data is %s'%str(test_data.shape))

In [ ]:
# convert operation ID to labels (from 0 to n)
labels = np.unique(train_data[:, -1])
label_dict = dict(zip(labels, np.arange(len(labels))))
train_data[:,-1] = np.array([label_dict[i] for i in train_data[:,-1]])
val_data[:,-1] =  np.array([label_dict[i] for i in val_data[:,-1]])
test_data[:,-1] =  np.array([label_dict[i] for i in test_data[:,-1]])

## 3.2 Prepare Dataloader

---

## 3.2 データローダーの準備

In [ ]:
class data_loader_OpenPack(Dataset):
    def __init__(self, samples, labels, device='cpu'):
        self.samples = torch.tensor(samples).to(device)  # check data type
        self.labels = torch.tensor(labels)  # check data type

    def __getitem__(self, index):
        target = self.labels[index]
        sample = self.samples[index]
        return sample, target

    def __len__(self):
        return len(self.labels)

def sliding_window(datanp, len_sw, step):
    '''
    :param datanp: shape=(data length, dim) raw sensor data and the labels. The last column is the label column.
    :param len_sw: length of the segmented sensor data
    :param step: overlapping length of the segmented data
    :return: shape=(N, len_sw, dim) batch of sensor data segment.
    '''

    # generate batch of data by overlapping the training set
    data_batch = []
    for idx in range(0, datanp.shape[0] - len_sw - step, step):
        data_batch.append(datanp[idx: idx + len_sw, :])
    data_batch.append(datanp[-1 - len_sw: -1, :])  # last batch
    xlist = np.stack(data_batch, axis=0)  # [B, data length, dim]

    return xlist

def generate_dataloader(data, len_sw, step, if_shuffle=True):
    tmp_b = sliding_window(data, len_sw, step)
    data_b = tmp_b[:, :, :-1]
    label_b = tmp_b[:, :, -1]
    data_set_r = data_loader_OpenPack(data_b, label_b, device=device)
    data_loader = DataLoader(data_set_r, batch_size=batch_size,
                              shuffle=if_shuffle, drop_last=False)
    return data_loader

In [ ]:
len_sw = 300
step = 150
batch_size = 512

train_loader = generate_dataloader(train_data, len_sw, step, if_shuffle=True)
val_loader = generate_dataloader(val_data, len_sw, step, if_shuffle=False)
test_loader = generate_dataloader(test_data, len_sw, step, if_shuffle=False)

## 3.3 Prepare Model

Reference:
https://github.com/Tian0426/CL-HAR

---
## 3.3 モデルの準備

参照:
https://github.com/Tian0426/CL-HAR


In [ ]:
class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, **kwargs):
        return self.fn(x, **kwargs) + x

class Attention(nn.Module):
    def __init__(self, dim, heads=8, dropout=0.5):
        super().__init__()
        self.heads = heads
        self.scale = dim ** -0.5

        self.to_qkv = nn.Linear(dim, dim * 3, bias=False)
        self.to_out = nn.Sequential(
            nn.Linear(dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x, mask=None):
        b, n, _, h = *x.shape, self.heads
        qkv = self.to_qkv(x).chunk(3, dim=-1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h=h), qkv)

        dots = torch.einsum('bhid,bhjd->bhij', q, k) * self.scale

        if mask is not None:
            mask = F.pad(mask.flatten(1), (1, 0), value=True)
            assert mask.shape[-1] == dots.shape[-1], 'mask has incorrect dimensions'
            mask = mask[:, None, :] * mask[:, :, None]
            dots.masked_fill_(~mask, float('-inf'))
            del mask

        self.attn = dots.softmax(dim=-1)

        out = torch.einsum('bhij,bhjd->bhid', self.attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        out = self.to_out(out)
        return out

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout=0.5):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn

    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class Transformer_block(nn.Module):
    def __init__(self, dim, depth, heads, mlp_dim, dropout):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                (PreNorm(dim, Attention(dim, heads=heads, dropout=dropout))),
                Residual(PreNorm(dim, FeedForward(dim, mlp_dim, dropout=dropout)))
            ]))

    def forward(self, x, mask=None):
        for attn, ff in self.layers:
            x = attn(x, mask=mask)
            x = ff(x)
        return x

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], requires_grad=False)
        return self.dropout(x)

class Seq_Transformer(nn.Module):
    def __init__(self, n_channel, len_sw, n_classes, dim=128, depth=4, heads=4, mlp_dim=64, dropout=0.1):
        super().__init__()
        self.patch_to_embedding = nn.Linear(n_channel, dim)
        self.c_token = nn.Parameter(torch.randn(1, 1, dim))
        self.position = PositionalEncoding(d_model=dim, max_len=len_sw)
        self.transformer = Transformer_block(dim, depth, heads, mlp_dim, dropout)
        self.to_c_token = nn.Identity()
        self.classifier = nn.Linear(dim, n_classes)


    def forward(self, forward_seq):
        x = self.patch_to_embedding(forward_seq)
        x = self.position(x)
        b, n, _ = x.shape
        c_tokens = repeat(self.c_token, '() n d -> b n d', b=b)
        x = torch.cat((c_tokens, x), dim=1)
        x = self.transformer(x)
        c_t = self.to_c_token(x[:, 0])
        return c_t

class Transformer(nn.Module):
    def __init__(self, n_channels=6, len_sw=300, n_classes=11, dim=128, depth=4, heads=4, mlp_dim=64, dropout=0.3):
        super(Transformer, self).__init__()

        self.out_dim = dim
        self.transformer = Seq_Transformer(n_channel=n_channels, len_sw=len_sw, n_classes=n_classes, dim=dim, depth=depth, heads=heads, mlp_dim=mlp_dim, dropout=dropout)
        self.classifier = nn.Linear(dim, n_classes)

    def forward(self, x):
        x = self.transformer(x)
        out = self.classifier(x)
        return out
        # return out, x

In [ ]:
model = Transformer()
model = model.to(device)
print(model)

## 3.4 Training and test

Reference:
https://github.com/jhhuang96/ConvLSTM-PyTorch/blob/master/main.py

---
## 3.4 トレーニングとテスト

参考:
https://github.com/jhhuang96/ConvLSTM-PyTorch/blob/master/main.py


In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                if self.verbose:
                    print("Early stopping triggered")
                return True
        return False
early_stopping = EarlyStopping()

In [ ]:
def vote_labels(label):
    # Iterate over each sample in the batch
    votel = []
    for i in range(label.size(0)):
        # Get unique labels and their counts
        unique_labels, counts = label[i].unique(return_counts=True)

        # Find the index of the maximum count
        max_count_index = counts.argmax()

        # Get the label corresponding to that maximum count
        mode_label = unique_labels[max_count_index]

        # Append the mode to the result list
        votel.append(mode_label)

    # Convert the result list to a tensor and reshape to (batch, 1)
    vote_label = torch.tensor(votel, dtype=torch.long).view(-1)
    return vote_label

In [ ]:
num_epochs = 100

criterion = torch.nn.CrossEntropyLoss()
criterion = criterion.to(device)

learning_rate = 0.001
optimizer = optim.Adam(
            model.parameters(), lr=learning_rate, amsgrad=True
        )

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

In [ ]:
train_losses, val_losses = [], []
for epoch in tqdm(range(num_epochs)):
    train_loss, val_loss = [], []
    ###################
    # train the model #
    ###################
    model.train()
    true_labels, pred_labels = [], []
    for i, (sample, label) in enumerate(train_loader):
        sample = sample.to(device=device, dtype=torch.float)
        label = label.to(device=device, dtype=torch.long)
        vote_label = vote_labels(label)
        vote_label = vote_label.to(device)
        output = model(sample)  # x_encoded.shape=batch512,outchannel128,len13
        loss = criterion(output, vote_label)
        train_loss.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        true_labels.append(vote_label.detach().cpu().numpy())
        pred_labels.append(output.detach().cpu().numpy())

    train_losses.append(np.average(train_loss))
    # Calculate F1 scores
    y_true = np.concatenate(true_labels, axis=0)
    y_prob = np.concatenate(pred_labels, axis=0)

    # Get the predicted class labels (argmax along the class dimension)
    y_pred = np.argmax(y_prob, axis=1)  # output Shape: (batch_size, time_steps)

    # Calculate F1 score (macro F1 score)
    f1 = f1_score(y_true, y_pred, average='macro')

    print(f'F1 Score of training set: {f1:.4f}')

    ######################
    # validate the model #
    ######################
    with torch.no_grad():
        model.eval()
        true_labels, pred_labels = [], []
        for i, (sample, label) in enumerate(val_loader):
            sample = sample.to(device=device, dtype=torch.float)
            label = label.to(device=device, dtype=torch.long)
            vote_label = vote_labels(label)
            vote_label = vote_label.to(device)
            output = model(sample)
            loss = criterion(output, vote_label)
            val_loss.append(loss.item())
            true_labels.append(vote_label.detach().cpu().numpy())
            pred_labels.append(output.detach().cpu().numpy())
        val_losses.append(np.average(val_loss))

        # Calculate F1 scores
        y_true = np.concatenate(true_labels, axis=0)
        y_prob = np.concatenate(pred_labels, axis=0)

        # Get the predicted class labels (argmax along the class dimension)
        y_pred = np.argmax(y_prob, axis=1)  # output Shape: (batch_size, time_steps)

        # Calculate F1 score (macro F1 score)
        f1 = f1_score(y_true, y_pred, average='macro')

        print(f'F1 Score of validation set: {f1:.4f}')

        # Check early stopping
        if early_stopping(np.average(val_losses)):
            print("Stopping at epoch %s." % str(epoch))
            break
    scheduler.step(np.average(val_loss))
    # Print the current learning rate
    current_lr = scheduler.get_last_lr()[0]  # Get the current learning rate
    print(f'Epoch {epoch + 1}, Learning Rate: {current_lr}')

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(val_losses, label='valid loss')
plt.plot(train_losses, label='train loss')
plt.grid()
plt.legend()
plt.xlabel('iteration')
plt.ylabel('loss value')
plt.show()

In [ ]:
with torch.no_grad():
    model.eval()
    true_labels, pred_labels = [], []
    for i, (sample, label) in enumerate(test_loader):
        sample = sample.to(device=device, dtype=torch.float)
        label = label.to(device=device, dtype=torch.long)
        vote_label = vote_labels(label)
        # vote_label = vote_label.to(device)
        output = model(sample)  # x_encoded.shape=batch512,outchannel128,len13

        true_labels.append(vote_label.numpy())
        pred_labels.append(output.detach().cpu().numpy())

    # Calculate F1 scores
    y_true = np.concatenate(true_labels, axis=0)
    y_prob = np.concatenate(pred_labels, axis=0)

    # Get the predicted class labels (argmax along the class dimension)
    y_pred = np.argmax(y_prob, axis=1)  # output Shape: (batch_size, time_steps)

    # Calculate F1 score (macro F1 score)
    f1 = f1_score(y_true, y_pred, average='macro')

    print(f'F1 Score of test set: {f1:.4f}')

# 4. Submission

Participants only need to submit the code related to the virtual data generation.
During the evaluation of the challenge, we will call "custom_virtual_data_generation" function to generate virtual data. So, please ensure that all relevant codes are included in the .py file.



---
# 4. 提出

参加者は仮想データ生成に関連するコードのみを提出する必要があります。
チャレンジの評価中に、仮想データを生成するために「custom_virtual_data_generation」関数を呼び出します。ここでは、関連するすべてのコードが .py ファイルに含まれていることを確認してください。


In [ ]:
# Define the functions you want to save
functions_code = """
import numpy as np

def switch_axis(sample, choice):

    x = sample[0, :]
    y = sample[1, :]
    z = sample[2, :]

    if choice == 0:
        return sample
    elif choice == 1:
        sample = np.stack([x, y, z], axis=0)
    elif choice == 2:
        sample = np.stack([x, z, y], axis=0)
    elif choice == 3:
        sample = np.stack([y, x, z], axis=0)
    elif choice == 4:
        sample = np.stack([y, z, x], axis=0)
    elif choice == 5:
        sample = np.stack([z, x, y], axis=0)
    elif choice == 6:
        sample = np.stack([z, y, x], axis=0)
    return sample


def flip(sample, choice):

    if choice == 1:
        sample = np.flip(sample, 1)
    return sample


def DA_Permutation(X, nPerm=4, minSegLength=10):
    X_new = np.zeros(X.shape)
    idx = np.random.permutation(nPerm)
    bWhile = True
    while bWhile is True:
        segs = np.zeros(nPerm + 1, dtype=int)
        segs[1:-1] = np.sort(
            np.random.randint(
                minSegLength, X.shape[0] - minSegLength, nPerm - 1
            )
        )
        segs[-1] = X.shape[0]
        if np.min(segs[1:] - segs[0:-1]) > minSegLength:
            bWhile = False
    pp = 0
    for ii in range(nPerm):
        x_temp = X[segs[idx[ii]] : segs[idx[ii] + 1], :]
        X_new[pp : pp + len(x_temp), :] = x_temp
        pp += len(x_temp)
    return X_new


def permute(sample, choice, nPerm=4, minSegLength=10):

    if choice == 1:
        sample = np.swapaxes(sample, 0, 1)
        sample = DA_Permutation(sample, nPerm=nPerm, minSegLength=minSegLength)
        sample = np.swapaxes(sample, 0, 1)
    return sample


def is_scaling_factor_invalid(scaling_factor, min_scale_sigma):

    for i in range(len(scaling_factor)):
        if abs(scaling_factor[i] - 1) < min_scale_sigma:
            return True
    return False


def DA_Scaling(X, sigma=0.3, min_scale_sigma=0.05):
    scaling_factor = np.random.normal(
        loc=1.0, scale=sigma, size=(1, X.shape[1])
    )  # shape=(1,3)
    while is_scaling_factor_invalid(scaling_factor, min_scale_sigma):
        scaling_factor = np.random.normal(
            loc=1.0, scale=sigma, size=(1, X.shape[1])
        )
    my_noise = np.matmul(np.ones((X.shape[0], 1)), scaling_factor)
    X = X * my_noise
    return X


def scaling_uniform(X, scale_range=0.15, min_scale_diff=0.02):
    low = 1 - scale_range
    high = 1 + scale_range
    scaling_factor = np.random.uniform(
        low=low, high=high, size=(X.shape[1])
    )  # shape=(3)
    while is_scaling_factor_invalid(scaling_factor, min_scale_diff):
        scaling_factor = np.random.uniform(
            low=low, high=high, size=(X.shape[1])
        )

    for i in range(3):
        X[:, i] = X[:, i] * scaling_factor[i]

    return X


def scale(sample, choice, scale_range=0.5, min_scale_diff=0.15):
    if choice == 1:
        sample = np.swapaxes(sample, 0, 1)
        sample = scaling_uniform(
            sample, scale_range=scale_range, min_scale_diff=min_scale_diff
        )
        sample = np.swapaxes(sample, 0, 1)
    return sample


def DistortTimesteps(X, sigma=0.2):
    tt = GenerateRandomCurves(
        X, sigma
    )  # Regard these samples aroun 1 as time intervals
    tt_cum = np.cumsum(tt, axis=0)  # Add intervals to make a cumulative graph
    # Make the last value to have X.shape[0]
    t_scale = [
        (X.shape[0] - 1) / tt_cum[-1, 0],
        (X.shape[0] - 1) / tt_cum[-1, 1],
        (X.shape[0] - 1) / tt_cum[-1, 2],
    ]
    tt_cum[:, 0] = tt_cum[:, 0] * t_scale[0]
    tt_cum[:, 1] = tt_cum[:, 1] * t_scale[1]
    tt_cum[:, 2] = tt_cum[:, 2] * t_scale[2]
    return tt_cum


def GenerateRandomCurves(X, sigma=0.2, knot=4):
    xx = (
        np.ones((X.shape[1], 1))
        * (np.arange(0, X.shape[0], (X.shape[0] - 1) / (knot + 1)))
    ).transpose()
    yy = np.random.normal(loc=1.0, scale=sigma, size=(knot + 2, X.shape[1]))
    x_range = np.arange(X.shape[0])
    cs_x = CubicSpline(xx[:, 0], yy[:, 0])
    cs_y = CubicSpline(xx[:, 1], yy[:, 1])
    cs_z = CubicSpline(xx[:, 2], yy[:, 2])
    return np.array([cs_x(x_range), cs_y(x_range), cs_z(x_range)]).transpose()


def DA_TimeWarp(X, sigma=0.2):
    tt_new = DistortTimesteps(X, sigma)
    X_new = np.zeros(X.shape)
    x_range = np.arange(X.shape[0])
    X_new[:, 0] = np.interp(x_range, tt_new[:, 0], X[:, 0])
    X_new[:, 1] = np.interp(x_range, tt_new[:, 1], X[:, 1])
    X_new[:, 2] = np.interp(x_range, tt_new[:, 2], X[:, 2])
    return X_new


def time_warp(sample, choice, sigma=0.2):
    if choice == 1:
        sample = np.swapaxes(sample, 0, 1)
        sample = DA_TimeWarp(sample, sigma=sigma)
        sample = np.swapaxes(sample, 0, 1)
    return sample

def custom_virtual_data_generation_algorithm(data):

     # Data augmentations
    left = permute(data[:,:3].transpose(), 1)
    right = time_warp(data[:,3:].transpose(), 1)

    new_data = np.concatenate([left.transpose(), right.transpose()], axis=1)
    return new_data

def save_virtual_data(data, filename):

  data.to_csv(os.path.join(virt_directory, filename+'.csv'), index=False)
  return

def custom_virtual_data_generation(train_data_dict):

  for u, df in train_data_dict.items():
    print('Generating virtual data from user %s.'% u)
    # Extract sensor data and labels
    raw_data = df[selected_columns[:6]].values
    labels = df[selected_columns[-1]].values.reshape(-1,1)

    tmp = custom_virtual_data_generation_algorithm(raw_data)

    # Concatenate data with operation labels
    virtual_data = np.concatenate([tmp, labels], axis=1)

    # Convert np.array to dataframe
    df = pd.DataFrame(virtual_data, columns=new_columns)

    # Save data to /data/virtual/
    save_virtual_data(df, u)
    # df.to_csv(os.path.join(virt_directory, u+'.csv'), index=False)

"""

# Create a new Python file and write the functions to it
with open(rootdir+'/custom_functions.py', 'w') as file:
    file.write(functions_code)